# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
# join the file path and roots with the subdirectories using glob
for root, dirs, files in os.walk(filepath):
    file_path_list = glob.glob(os.path.join(root,'*'))


/home/workspace/Data-Engineer-Project/Project 2-Data Modeling with Apache Cassandra


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
for f in file_path_list:
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        for line in csvreader:
            full_data_rows_list.append(line) 

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [5]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster(['127.0.0.1'])

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [6]:
# Create a Keyspace 
query = """CREATE KEYSPACE IF NOT EXISTS cassandra_table
           WITH REPLICATION = { 'class' : 'SimpleStrategy', 'replication_factor' : 1}
        """

try:
    session.execute(query)
except Exception as e:
    print(e)

#### Set Keyspace

In [7]:
# Set KEYSPACE to the keyspace specified above
session.set_keyspace('cassandra_table')

## Create queries to ask the following three questions of the data

### 1. return the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4

### 2. return only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

### 3. return every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




### Note: The sequence of the columns in the CREATE and INSERT statements `MUST` follow the order of the PRIMARY KEY and CLUSTERING columns. 

In [8]:
## Query 1:  Give me the artist, song title and song's length in the music app history that was heard during \
## sessionId = 338, and itemInSession = 4
create_query_1 = """CREATE TABLE IF NOT EXISTS item (
                    sessionId INT, 
                    itemInSession INT, 
                    artist TEXT, 
                    song TEXT, 
                    length FLOAT, 
                    PRIMARY KEY (sessionId, itemInSession)
                )"""
try:
    session.execute(create_query_1)
except Exception as e:
    print(e)

In [9]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO item (sessionId, itemInSession, artist, song, length)"
        query = query + "VALUES (%s, %s, %s, %s, %s)"
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[10], float(line[5])))

#### Do a SELECT to verify that the data have been inserted into each table

In [10]:
## Add in the SELECT statement to verify the data was entered into the table
select_query_1 = """SELECT artist, song, length
                    FROM item
                    WHERE sessionId = 338
                    AND itemInSession = 4
                 """
try:
    queries = session.execute(select_query_1)
except Exception as e:
    print(e)
    
for query in queries:
    print(query)

Row(artist='Faithless', song='50', length=495.30731201171875)


### COPY AND REPEAT THE ABOVE THREE CELLS FOR EACH OF THE THREE QUESTIONS

In [13]:
## Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name)\
## for userid = 10, sessionid = 182
create_query_2 = """CREATE TABLE IF NOT EXISTS user (sessionId int, userId int, itemInSession int, artist text, song text, firstName text, lastName text, 
                                                     PRIMARY KEY ((sessionId, userId), itemInSession))
                    WITH CLUSTERING ORDER BY (itemInSession ASC)
                 """
try:
    session.execute(create_query_2)
except Exception as e:
    print(e)

file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO user (sessionId, userId, itemInSession, artist, song, firstName, lastName)"
        query = query + "VALUES (%s, %s, %s, %s, %s, %s, %s)"
        session.execute(query, (int(line[8]), int(line[10]), int(line[3]), line[0], line[9], line[1], line[4]))

In [15]:
select_query_2 = """SELECT artist, song, firstName, lastName 
                   FROM  user
                   WHERE sessionId = 182 
                   AND userId = 10
                """
try:
    queries = session.execute(select_query_2)
except Exception as e:
    print(e)

for query in queries:
    print(query)

Row(artist='Down To The Bone', song="Keep On Keepin' On", firstname='Sylvie', lastname='Cruz')
Row(artist='Three Drives', song='Greece 2000', firstname='Sylvie', lastname='Cruz')
Row(artist='Sebastien Tellier', song='Kilometer', firstname='Sylvie', lastname='Cruz')
Row(artist='Lonnie Gordon', song='Catch You Baby (Steve Pitron & Max Sanna Radio Edit)', firstname='Sylvie', lastname='Cruz')


In [16]:
## Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'
create_query_3 = """CREATE TABLE IF NOT EXISTS song_table (song TEXT, userId INT, firstName TEXT, lastName TEXT, PRIMARY KEY ((song), userId))"""

try:
    session.execute(create_query_3)
except Exception as e:
    print(e)

file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO song_table (song, userId, firstName, lastName)"
        query = query + "VALUES (%s, %s, %s, %s)"
        session.execute(query, (line[9], int(line[10]), line[1], line[4]))
                    

In [17]:
select_query_3 = """SELECT song, firstName, lastName 
                    FROM song_table 
                    WHERE song = 'All Hands Against His Own'
                 """
try:
    queries = session.execute(select_query_3)
except Exception as e:
    print(e)

for query in queries:
    print(query)

Row(song='All Hands Against His Own', firstname='Jacqueline', lastname='Lynch')
Row(song='All Hands Against His Own', firstname='Tegan', lastname='Levine')
Row(song='All Hands Against His Own', firstname='Sara', lastname='Johnson')


### Drop the tables before closing out the sessions

In [18]:
## Drop the table before closing out the sessions
session.execute("DROP TABLE IF EXISTS cassandra_table.item")
session.execute("DROP TABLE IF EXISTS cassandra_table.user")
session.execute("DROP TABLE IF EXISTS cassandra_table.song_table")

### Close the session and cluster connection¶

In [19]:
session.shutdown()
cluster.shutdown()